In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import random as rand

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
import os
os.chdir('../../')

In [3]:
from programs import lstm_tool

In [4]:
df = pd.read_excel('./documents/nov_nine_var.xlsx').iloc[:,1::].to_numpy()

scaler = MinMaxScaler()
scale_df = scaler.fit_transform(df[:,0:21])

In [5]:
def noiser(data):
    data[rand.randint(0, len(data)-1)] = data[rand.randint(0, len(data)-1)]
    return data

In [6]:
def shuffler(data):
    length = 200
    idx = rand.randint(0, len(data)-length)
    data = data[idx:idx+length]

    return data

In [7]:
real_state = []
fake_state = []

for i in range(1000):
    # choose type--------------
    s = shuffler(scale_df)
    # s = scale_df
    # -------------------------

    real_state.append(s)
    fake_state.append(noiser(s))

state = np.concatenate((np.array(real_state), np.array(fake_state)))
label = np.concatenate((np.ones((len(real_state), 1)), np.ones((len(fake_state), 1))))

In [8]:
state_train, state_test, label_train, label_test = train_test_split(state, label, test_size=0.2, shuffle=True)

In [9]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(512, activation='tanh', input_shape=(200, 21), kernel_initializer=tf.keras.initializers.GlorotUniform(), return_sequences=True))
model.add(tf.keras.layers.LSTM(512, activation='tanh', kernel_initializer=tf.keras.initializers.GlorotUniform(), return_sequences=True))
model.add(tf.keras.layers.LSTM(256, activation='tanh', kernel_initializer=tf.keras.initializers.GlorotUniform(), return_sequences=True))
model.add(tf.keras.layers.LSTM(256, activation='tanh', kernel_initializer=tf.keras.initializers.GlorotUniform(), return_sequences=False))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')

In [10]:
state_test.shape

(400, 200, 21)

In [11]:
model.fit(state_train, label_train, epochs=100, 
        batch_size=256, validation_data=(state_test, label_test), verbose=1)

Epoch 1/100
7/7 [==============================] - 114s 14s/step - loss: 0.6902 - accuracy: 0.3600 - val_loss: 0.4812 - val_accuracy: 1.0000
Epoch 2/100
7/7 [==============================] - 84s 12s/step - loss: 0.2536 - accuracy: 1.0000 - val_loss: 0.0199 - val_accuracy: 1.0000
Epoch 3/100
7/7 [==============================] - 83s 12s/step - loss: 0.0060 - accuracy: 1.0000 - val_loss: 1.1438e-04 - val_accuracy: 1.0000
Epoch 4/100
7/7 [==============================] - 82s 12s/step - loss: 3.8922e-05 - accuracy: 1.0000 - val_loss: 2.5861e-06 - val_accuracy: 1.0000
Epoch 5/100
7/7 [==============================] - 81s 12s/step - loss: 1.1917e-06 - accuracy: 1.0000 - val_loss: 2.6229e-07 - val_accuracy: 1.0000
Epoch 6/100
7/7 [==============================] - 85s 12s/step - loss: 1.6114e-07 - accuracy: 1.0000 - val_loss: 7.2531e-08 - val_accuracy: 1.0000
Epoch 7/100
7/7 [==============================] - 96s 14s/step - loss: 5.4868e-08 - accuracy: 1.0000 - val_loss: 3.6362e-08 - val_

In [12]:
model.save('./model/l_checker.h5')